In [1]:
from transformers import TFAutoModelForMaskedLM, DistilBertTokenizerFast
from datasets import load_from_disk

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = DistilBertTokenizerFast.from_pretrained('test-tokenizer-cleaned-52000')
# tokenized_datasets = load_from_disk('distilbert-model/tokenized-dataset')
# lm_datasets = load_from_disk('distilbert-model/lm_dataset')
# dataset = load_from_disk("web-crawl-mc4")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [3]:
model.summary()


Model: "tf_distil_bert_for_masked_lm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
________________________

In [5]:
from datasets import load_dataset

dataset = load_dataset("mc4", "ka")
dataset

Found cached dataset mc4 (C:/Users/Davit6174/.cache/huggingface/datasets/mc4/ka/0.0.0/99acea4a740b4cc36e4a93a238c7de11b0ce341d65b7d37168b3b90fd64721d2)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'timestamp', 'url'],
        num_rows: 2295551
    })
    validation: Dataset({
        features: ['text', 'timestamp', 'url'],
        num_rows: 2279
    })
})

In [27]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> text: {row['text']}'")
    print(f"'>>> timestamp: {row['timestamp']}'")
    print(f"'>>> url: {row['url']}'")

Loading cached shuffled indices for dataset at C:\Users\Davit6174\.cache\huggingface\datasets\mc4\ka\0.0.0\99acea4a740b4cc36e4a93a238c7de11b0ce341d65b7d37168b3b90fd64721d2\cache-a044b8f1b0f877ce.arrow



'>>> text: გამოქვეყნებულია: 2011-07-13, 20:05
თემა: რამდენად უსაფრთხოა სინთეტიური სარეცხი საშუალებები ადამიანისთვის და გარემოსთვის.
სტუმრები: ქიმიკოსი ინგა ღვინერია, კომპანია " ბერტას " ტექნოლოგი, ხარისხის მართვის სამსახურის უფროსი გიორგი ჯირკვალიძე.'
'>>> timestamp: 2020-07-09T07:56:51Z'
'>>> url: http://old.radio1.ge/ge/videos/view/125602.html'

'>>> text: საჯარო დაწესებულება: აბაშის გამგეობა ნინოწმინდის გამგეობა ქალაქ გორის მერია ქალაქ ზუგდიდის მერია გორის მუნიციპალიტეტის გამგეობა ზესტაფონის გამგეობა თელავის მუნიციპალიტეტის გამგეობა სენაკის გამგეობა'
'>>> timestamp: 2020-03-29T06:25:57Z'
'>>> url: https://www.datalab.ge/ge/data/?cl=public_institutions&public_institutions=129_178_115_124_104_141_142_132&tags=58'

'>>> text: ბედისწერას ვერ გაექცევი! - ცნობილი ვარსკვლავების დაუჯერებელი სასიყვარულო ისტორიები - Primetime.Ge
ბედისწერას ვერ გაექცევი! - ცნობილი ვარსკვლავების დაუჯერებელი სასიყვარულო ისტორიები
16:36 03-14-2019
თეგები: ნიკოლას კეიჯიმეთ დეიმონირობერტ დე ნიროალექს ბოლდუინი
საზო

In [ ]:
from transformers import AutoTokenizer


def get_training_corpus():
    train_dataset = dataset["train"]
    for start_idx in range(0, len(train_dataset), 1000):
        samples = train_dataset[start_idx : start_idx + 1000]
        yield samples["text"]


training_corpus = get_training_corpus()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
new_tokenizer = tokenizer.train_new_from_iterator(training_corpus, 30000)
new_tokenizer.save_pretrained("distilbert-tokenizer")

In [29]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "timestamp", "url"]
)
tokenized_datasets.save_to_disk('distilbert-model/tokenized-dataset')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 1836440
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 459111
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 2279
    })
})

In [2]:
chunk_size = 512

tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> ტექსტი {idx} სიგრძე: {len(sample)}'")

'>>> ტექსტი 0 სიგრძე: 878'
'>>> ტექსტი 1 სიგრძე: 18'
'>>> ტექსტი 2 სიგრძე: 459'


In [33]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> გაერთიანებული ტქსტების სიგრძე: {total_length}'")

'>>> გაერთიანებული ტქსტების სიგრძე: 1355'


In [34]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> დანაწევრებული ზომა: {len(chunk)}'")

'>>> დანაწევრებული ზომა: 512'
'>>> დანაწევრებული ზომა: 512'
'>>> დანაწევრებული ზომა: 331'


In [35]:
def group_texts(examples):
    # შევაწებოთ ტექსტები
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # გამოვთვალოთ გაერთიანებული მონაცემების სიგრძე
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # თუ სხვებზე პატარაა ბოლო ნაწილი მოდელში არ შევუშვათ
    total_length = (total_length // chunk_size) * chunk_size
    # მითითებული სიგრძის მიხედვით დავანაწევროდ
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # შევქმნათ ახალი სვეტი labels
    result["labels"] = result["input_ids"].copy()
    return result

In [36]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets.save_to_disk('distilbert-model/lm_dataset')
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3346701
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 836770
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 4089
    })
})

In [2]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [17]:
samples = [lm_datasets["train"][i] for i in range(3997,4000)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> ადგილი ; ● [MASK]ადრაკში - მარიკა ბიბილური - მეორე ადგილი ქალთა შორის და ვანო [MASK]ომიძე ( რეალიზაციის განყოფილება ) [MASK] მესამე ადგილი მამაკაც [MASK] შორის [MASK] ასე რომ [MASK] ჟურნალი „ ავერსი “ მკითხ 嘛 ჯანსაღი ცხოვრებისკენ არა [MASK] ჟურნალის ფურცლებიდან, არამედ პირადი მაგალითითა [MASK] აქტიურად მოუწოდებს. [SEP] [CLS] შატილი | არაფერი პირადი არაფერი პირადი post factumm archive for the tag “ შატილი ” 11 share this : facebookt [MASK]itterlike this [MASK] like იტვირთება... posted in my h [MASK]ory [MASK] tagged ბარისახო [MASK] [MASK]ქო, მუც [MASK], ფშავი [MASK] შატილი | დატოვე კომენტარი post navigation კატეგორიები [SEP] [CLS] ნანადირევი - ჩვენი ნანადირევი - მონ [MASK]ე [MASK] ფოტოალბომი - მონ [MASK]ეების საიტი მთავარი » ფოტოალბ [MASK] » მონადირე » ჩვენი ნანადირევი [MASK] ნანადირევი ნანახია : 1876 | [MASK] [MASK] : 960 [MASK]5 [MASK]px / [MASK]3. 6kb თარიღი :ᛕ. 12. 2015 | დაამატა : muri [MASK]o6 « [MASK] [MASK] 9 [MASK] 92 93 94 95 [ 96 ] 97 98 99 100 101 | შემდეგი » [SEP] [CL

In [18]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")
        # სიტყვებსა და ტოკენებს შორის კავშირების დამყარება
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # სიტყვების შემთხვევითად შენიღბვა
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [70]:
samples = [lm_datasets["train"][i] for i in range(3997,4000)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [MASK] ; ● ჭადრაკში - მარიკა ბიბილური [MASK] მეორე ადგილი ქალთა [MASK] და ვანო ლომიძე ( რეალიზაციის განყოფილება ) - მესამე ადგილი მამაკაცთა შორის. ასე რომ, ჟურნალი „ [MASK] [MASK] [MASK] [MASK] მკითხველს ჯანსაღი ცხოვრებისკენ არა მხოლოდ ჟურნალის ფურცლებიდან, [MASK] [MASK] [MASK] [MASK] [MASK] აქტიურად მოუწოდებს. [SEP] [CLS] შატილი | არაფერი პირადი არაფერი პირადი post factumm archive for the [MASK] “ შატილი ” [MASK] share this : facebooktwitterlike this : [MASK] [MASK] იტვირთება [MASK].. posted in my [MASK] [MASK] [MASK] and tagged [MASK] [MASK] [MASK] [MASK], მუქო, [MASK] [MASK] [MASK], ფშავი, შატილი | [MASK] [MASK] კომენტარი post navigation [MASK] [MASK] [SEP] [CLS] ნანადირევი [MASK] ჩვენი ნანადირევი - მონადირე - ფოტოალბომი - [MASK] [MASK] [MASK] საიტი მთავარი » ფოტოალბომი » მონადირე » ჩვენი ნანადირევი » ნანადირევი [MASK] : 1876 | ზომები : 960x540px / [MASK] [MASK]. 6kb თარიღი [MASK] 12 [MASK] 12. 2015 | დაამატა [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] წინა | 91 92 93 [MA

In [7]:
tf_train_dataset = model.prepare_tf_dataset(
    lm_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    lm_datasets["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

In [4]:
from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090, compute capability 8.9


In [5]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

6538/6538 [==============================] - 3207s 490ms/step - loss: 13.8668
Perplexity: 1052587.69


In [8]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset)

 27750/104584 [======>.......................] - ETA: 7:03:01 - loss: 3.6450

In [9]:
model.save_pretrained('distilbert-base-uncased-fine-tuned')

In [10]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model=model
)

In [15]:
preds = mask_filler("[MASK], როგორ ხარ?")
print('hello')
for pred in preds:
    print(f">>> {pred['sequence']}")